# Part 1 - Combine Course Info with Requirements

### 1. Read the course schedule into a DataFrame

In [90]:
from bs4 import BeautifulSoup
import re
import pandas as pd

with open('courseSchedule.html', 'r', encoding = "utf-8") as f:

    contents = f.read()

    soup = BeautifulSoup(contents, "html.parser")

courses = []
names = []
instructors = []
times = []

class_li = soup.find_all('li', {'class': 'row'})
for course in class_li :
    courses.append(course.span.text.replace('\u200b', '').strip().split('\n')[0])
    name = course.find('a', {'class': 'expand'}).get_text().strip().replace("\n", "")
    repair_name = re.sub(' +', ' ', name)
    names.append(repair_name)

    instructor_and_time = course.find_all("span", {'class': 'col-xs-12 col-sm-2'})
    each_instructor = instructor_and_time[0].get_text().strip().replace("\n", "")
    clean_instructors = re.sub(' +', ' ', each_instructor)
    instructors.append(clean_instructors)
    each_time = instructor_and_time[1].get_text().strip().replace("\n", "")
    times.append(each_time)

df = pd.DataFrame({'Number-Section': courses, 'Name': names, 'Instructor': instructors, 'Time': times})
separated_num_sec = pd.Series(courses).str.extract(r'(\w+.\w+.\d{4}).(\d{3})')
df["Number"] = separated_num_sec[0]
df["Section"] = separated_num_sec[1]
df.info()
df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  151 non-null    object
 1   Name            151 non-null    object
 2   Instructor      151 non-null    object
 3   Time            151 non-null    object
 4   Number          151 non-null    object
 5   Section         151 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


,Number-Section,Name,Instructor,Time,Number,Section
0,CSCI-GA.1170-001,Fundamental Algorithms,Yevgeniy Dodis,T 7:10-9:10PM,CSCI-GA.1170,001
1,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Aditya Pandey,R 8:10-9:00PM,CSCI-GA.1170,002
2,CSCI-GA.1180-001,Mathematical Techniques For CS Applications,Parijat Dube,W 7:10-9:10PM,CSCI-GA.1180,001
3,CSCI-GA.2110-001,Programming Languages,Cory Plock,M 4:55-6:55PM,CSCI-GA.2110,001
4,CSCI-GA.2110-002,Programming Languages Recitation,Jahnavi Pothineni,R 7:10-8:00PM,CSCI-GA.2110,002


In [91]:
df.tail(5)

,Number-Section,Name,Instructor,Time,Number,Section
146,CSCI-UA.0480-061,Special Topics: Open Source Software Development,Joanna Klukowska,MW 12:30-1:45PM,CSCI-UA.0480,061
147,CSCI-UA.0480-063,Special Topics: Introduction to Computer Security,Joseph Bonneau,MW 2:00-3:15PM,CSCI-UA.0480,063
148,CSCI-UA.0480-069,Special Topics: Agile Software Development and...,Amos Bloomberg,MW 12:30-1:45PM,CSCI-UA.0480,069
149,CSCI-UA.0480-072,Special Topics: Introduction to Robot Intellig...,Lerrel Pinto,TR 3:30-4:45PM,CSCI-UA.0480,072
150,CSCI-UA.0480-073,Special Topics: Randomized Algorithms,Richard Cole,MW 9:30-10:45AM,CSCI-UA.0480,073


In [92]:
df.sample(5)

,Number-Section,Name,Instructor,Time,Number,Section
1,CSCI-GA.1170-002,Fundamental Algorithms Recitation,Aditya Pandey,R 8:10-9:00PM,CSCI-GA.1170,002
39,CSCI-GA.3205-001,Applied Cryptography & Network Security,Mazdak Zamani,T 4:55-6:55PM,CSCI-GA.3205,001
67,CSCI-UA.0002-011,Intro To Computer Programming (No Prior Experi...,Matthew Zeidenberg,MW 3:30-4:45PM,CSCI-UA.0002,011
57,CSCI-UA.0002-001,Intro To Computer Programming (No Prior Experi...,Amanda Steigman,MW 8:00-9:15AM,CSCI-UA.0002,001
14,CSCI-GA.2566-001,Foundations Of Machine Learning*,Mehryar Mohri,T 4:55-6:55PM,CSCI-GA.2566,001


### 2. Read the course catalog into a DataFrame

In [93]:
from bs4 import BeautifulSoup
import re
import pandas as pd

with open('courseCatalog.html', 'r', encoding = "utf-8") as f:

    contents2 = f.read()

    soup2 = BeautifulSoup(contents2, "html.parser")

course_numbers = []
prereqs = []
credits = []
course_li = soup2.find_all('li', {'class': 'col-sm-12'})

for course in course_li:
    list = course.find_all('p')
    each_number = re.search('\w+-\w+.\d{4}', list[0].text.strip())
    course_numbers.append(each_number.group(0))

    preq_text = list[2].text.replace("Prerequisites:", "").strip()
    if '\xa0' in preq_text:
        preq_text = preq_text.replace(u'\xa0', u' ')
    prereqs.append(preq_text)

    cred_text = list[1].text.strip().split('\n')[0].replace('Points.',"").strip()
    credits.append(cred_text)

    
df2 = pd.DataFrame()
df2['Number'] = course_numbers
df2['Prereqs'] = prereqs
df2['Points'] = credits

df2.info()
df2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number   96 non-null     object
 1   Prereqs  96 non-null     object
 2   Points   96 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


,Number,Prereqs,Points
0,CSCI-GA.1133,None,4
1,CSCI-GA.1144,CSCI-GA 1133 or departmental permission.,4
2,CSCI-GA.1170,At least one year of experience with a high-le...,3
3,CSCI-GA.1180,None,3
4,CSCI-GA.2110,Students taking this class should already have...,3


In [94]:
df2.tail(5)

,Number,Prereqs,Points
91,CSCI-UA.0897,Restricted to declared computer science majors...,1 - 4
92,CSCI-UA.0898,Restricted to declared computer science majors...,1 - 4
93,CSCI-UA.0997,Permission of the department. Does not satisfy...,1 - 4
94,CSCI-UA.0998,Permission of the department. Does not satisfy...,1 - 4
95,FRSEM-UA.0597,"Some programming experience in Python, Java, J...",4


In [95]:
df2.sample(5)

,Number,Prereqs,Points
45,CSCI-GA.3033,Prerequisites vary according to topic.,3
31,CSCI-GA.2585,"Familiarity with basics in linear algebra, pro...",3
85,CSCI-UA.0478,Basic Algorithms (CSCI-UA 310).,4
21,CSCI-GA.2437,"Prerequisites include experience with Hadoop, ...",3
13,CSCI-GA.2274,"CSCI-GA 1170, CSCI-GA 2110, CSCI-GA 2250 and C...",3


### 3. Put together both DataFrames

In [96]:
df_merged = pd.merge(df, df2, on='Number', how='left')

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 150
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Number-Section  151 non-null    object
 1   Name            151 non-null    object
 2   Instructor      151 non-null    object
 3   Time            151 non-null    object
 4   Number          151 non-null    object
 5   Section         151 non-null    object
 6   Prereqs         151 non-null    object
 7   Points          151 non-null    object
dtypes: object(8)
memory usage: 10.6+ KB


In [97]:
df_merged = df_merged[ ['Number'] + [ col for col in df_merged.columns if col != 'Number' ] ]
del df_merged['Number-Section']
del df_merged['Section']
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 150
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Number      151 non-null    object
 1   Name        151 non-null    object
 2   Instructor  151 non-null    object
 3   Time        151 non-null    object
 4   Prereqs     151 non-null    object
 5   Points      151 non-null    object
dtypes: object(6)
memory usage: 8.3+ KB


In [98]:
df_merged.sample(5)

,Number,Name,Instructor,Time,Prereqs,Points
108,CSCI-UA.0201,Computer Systems Organization - Recitation,Yang Gao,F 3:30-4:45PM,Data Structures (CSCI-UA 102).,4
33,CSCI-GA.3033,Special Topics: Virtual Reality*,Kenneth Perlin,T 4:55-6:55PM,Prerequisites vary according to topic.,3
24,CSCI-GA.2830,The Lean Launch Pad Lab*,Yiannis Broustas,R 12:30-1:20PM,None,3
78,CSCI-UA.0061,Web Development And Programming,Craig Kapp,MW 12:30-1:45PM,Introduction to Computer Programming (No Prior...,4
137,CSCI-UA.0473,Fundamentals of Machine Learning - Recitation,Kaushik Tummalapalli,R 12:30-1:45PM,"Data Structures (CSCI-UA.102), Linear Algebra ...",4


### 4. Conclusion

Did you spot any anomalies, discrepancies, or unexpected data or relationships between data?

If so, in a markdown cell, describe any problem(s) you saw
additionally, describe how you might fix them (or if you already fixed them!)

Lastly, based on the resulting DataFrame, describe the behavior of how=left on these particular DataFrames

If you need to see all rows, use pd.set_option('display.max_rows', 200)

A lot of courses or credits were combined (MS and PhD) which I fixed by adding a column to label the type of education (whether it was undergrad, MS, or PhD). Some values were single digits, while others were presented as a range, and some ranges were categorized by MS and PhD. Inconsistencies also included extra spaces before and after the dashes or '-' symbol and other extra spaces with the data. To fix these discrepancies, one possible solution is to divide the points column into two separate columns: min credits and max credits. I used ranges to determine whether something was min or max credit. (Upper part of range = max, lower part = min). The min and max credits were filled in for the respective rows.

In part 3, the code merges two pandas data frames (df and df2) into a single data frame called df_merged based on a common column called "Number" using a left join method. The resulting df_merged data frame retains all the rows from the left data frame (df) and matching rows from the right data frame (df2), with any non-matching rows from df2 containing missing values. The info() method is then called on df_merged to print out summary information about the data frame such as the number of rows, columns, and data types of each column.
